In [ ]:
(require '[clojupyter.misc.helper :as helper])
(def _ (helper/add-dependencies '[org.scicloj/noj "2-beta4"]))

In [ ]:
(require '[tablecloth.api :as tc]
         '[clojure.string :as str]
         '[scicloj.kindly.v4.kind :as kind]
         '[scicloj.tableplot.v1.plotly :as plotly]
         '[scicloj.tableplot.v1.hanami :as hanami]
         '[scicloj.kindly-advice.v1.api :as kindly-advice]
         '[scicloj.kindly.v4.kind :as kind]
         '[scicloj.kindly.v4.api :as kindly]
         '[clojupyter.display :as display]
         '[cheshire.core :as cheshire]
         )

some helper functions

In [ ]:
(import javax.imageio.ImageIO
        java.net.URL)

(def image 
(kind/image
  (->  "https://upload.wikimedia.org/wikipedia/commons/e/eb/Ash_Tree_-_geograph.org.uk_-_590710.jpg"
       (java.net.URL.)
       (javax.imageio.ImageIO/read))))

(defn p [mark-size] 
    (-> iris
    (plotly/layer-point {:=x :sepal-width
                         :=y :sepal-length
                         :=color :species
                         :=mark-size mark-size})
    plotly/plot))


(defonce clay-image
  (->  "https://upload.wikimedia.org/wikipedia/commons/2/2c/Clay-ss-2005.jpg"
       (URL.)
       (ImageIO/read)))

(defn vega-lite-point-plot [data]
  (-> {:data {:values data},
       :mark "point"
       :encoding
       {:size {:field "w" :type "quantitative"}
        :x {:field "x", :type "quantitative"},
        :y {:field "y", :type "quantitative"},
        :fill {:field "z", :type "nominal"}}}
      kind/vega-lite))

(defn random-data [n]
  (->> (repeatedly n #(- (rand) 0.5))
       (reductions +)
       (map-indexed (fn [x y]
                      {:w (rand-int 9)
                       :z (rand-int 9)
                       :x x
                       :y y}))))

(defn random-vega-lite-plot [n]
  (-> n
      random-data
      vega-lite-point-plot))

(defn fetch-dataset [dataset-name]
  (-> dataset-name
      (->> (format "https://vincentarelbundock.github.io/Rdatasets/csv/%s.csv"))
      (tc/dataset {:key-fn (fn [k]
                             (-> k
                                 str/lower-case
                                 (str/replace #"\." "-")
                                 keyword))})
      (tc/set-dataset-name dataset-name)))
(defonce iris
  (fetch-dataset "datasets/iris"))

# Supported kinds
    

## :kind/dataset

In [ ]:
iris

## :kind/plotly

In [ ]:
(-> iris
    (plotly/layer-point {:=x :sepal-width
                         :=y :sepal-length
                         :=color :species
                         :=mark-size 10})
    plotly/plot)

In [ ]:
(def plotly-example
  {:data [{:x [0 1 3 2]
           :y [0 6 4 5]
           :z [0 8 9 7]
           :type :scatter3d
           :mode :lines+markers
           :opacity 0.5
           :line {:width 5}
           :marker {:size 4
                    :colorscale :Viridis}}]
   :layout {:title "Plotly example"}})

(kind/plotly
 plotly-example)

## :kind/md

In [ ]:
(kind/md "**bold**")

##  :kind/vega-lite

In [ ]:
(-> iris
    (hanami/plot hanami/point-chart
                 {:=x :sepal-width
                  :=y :sepal-length
                  :=color :species
                  :=mark-size 200}))

## :kind/tex

In [ ]:
(kind/tex "$\\sin{x}$")

## :kind/image

In [ ]:
(kind/image
  (->  "https://upload.wikimedia.org/wikipedia/commons/e/eb/Ash_Tree_-_geograph.org.uk_-_590710.jpg"
       (java.net.URL.)
       (javax.imageio.ImageIO/read)))

## :kind/html

In [ ]:
(kind/html
 "
<svg height=100 width=100>
<circle cx=50 cy=50 r=40 stroke='purple' stroke-width=3 fill='floralwhite' />
</svg> ")

## :kind/echarts


In [ ]:
(kind/echarts
 {:title {:text "Echarts Example"}
  :tooltip {}
  :legend {:data ["sales"]}
  :xAxis {:data ["Shirts", "Cardigans", "Chiffons",
                 "Pants", "Heels", "Socks"]}
  :yAxis {}
  :series [{:name "sales"
            :type "bar"
            :data [5 20 36
                   10 10 20]}]})

In [ ]:
^kind/tex ["$\\sqrt{x^2+1}$"]

In [ ]:
^kind/tex ["$\\sqrt{x^2+1}$" "$\\sqrt{x^2+1}$"]

In [ ]:
(kind/hiccup
 [:table [:tr [:td "$\\sqrt{x^2+1}$"]]
         [:tr [:td "$\\sin{x}$"]]])
 

## :kind/cytoscope


In [ ]:
(kind/cytoscape
 {:elements {:nodes [{:data {:id "a" :parent "b"} :position {:x 215 :y 85}}
                     {:data {:id "b"}}
                     {:data {:id "c" :parent "b"} :position {:x 300 :y 85}}
                     {:data {:id "d"} :position {:x 215 :y 175}}
                     {:data {:id "e"}}
                     {:data {:id "f" :parent "e"} :position {:x 300 :y 175}}]
             :edges [{:data {:id "ad" :source "a" :target "d"}}
                     {:data {:id "eb" :source "e" :target "b"}}]}
  :style [{:selector "node"
           :css {:content "data(id)"
                 :text-valign "center"
                 :text-halign "center"}}
          {:selector "parent"
           :css {:text-valign "top"
                 :text-halign "center"}}
          {:selector "edge"
           :css {:curve-style "bezier"
                 :target-arrow-shape "triangle"}}]
  :layout {:name "preset"
           :padding 5}})

## :kind/code

In [ ]:
(kind/code
 '(defn f [x] {:y (+  x 9)}))

## :kind/hidden

In [ ]:
(kind/hidden
 {:x 9})

## :kind/video

In [ ]:
(kind/video
 {:youtube-id "DAQnvAgBma8"})

## :kind/pprint

In [ ]:
(->> (range 30)
     (apply array-map)
     kind/pprint)

## :kind/fragment

In [ ]:
(->> ["purple" "darkgreen" "goldenrod"]
     (mapcat (fn [color]
               [(kind/md (str "### subsection: " color))
                (kind/hiccup [:div {:style {:background-color color
                                            :color "lightgrey"}}
                              [:big [:p color]]])]))
     kind/fragment)

## :kind/portal

In [ ]:
(kind/portal
 {:x (range 3)})

## :kind/highcharts

In [ ]:
(kind/highcharts
 {:title {:text "Line chart"}
  :subtitle {:text "By Job Category"}
  :yAxis {:title {:text "Number of Employees"}}
  :series [{:name "Installation & Developers"
            :data [43934, 48656, 65165, 81827, 112143, 142383,
                   171533, 165174, 155157, 161454, 154610]}

           {:name "Manufacturing",
            :data [24916, 37941, 29742, 29851, 32490, 30282,
                   38121, 36885, 33726, 34243, 31050]}

           {:name "Sales & Distribution",
            :data [11744, 30000, 16005, 19771, 20185, 24377,
                   32147, 30912, 29243, 29213, 25663]}

           {:name "Operations & Maintenance",
            :data [nil, nil, nil, nil, nil, nil, nil,
                   nil, 11164, 11218, 10077]}

           {:name "Other",
            :data [21908, 5548, 8105, 11248, 8989, 11816, 18274,
                   17300, 13053, 11906, 10073]}]

  :xAxis {:accessibility {:rangeDescription "Range: 2010 to 2020"}}

  :legend {:layout "vertical",
           :align "right",
           :verticalAlign "middle"}

  :plotOptions {:series {:label {:connectorAllowed false},
                         :pointStart 2010}}

  :responsive {:rules [{:condition {:maxWidth 500},
                        :chartOptions {:legend {:layout "horizontal",
                                                :align "center",
                                                :verticalAlign "bottom"}}}]}})

## :kind/fn

Partialy working


In [ ]:
(-> iris
    (plotly/layer-point {:=x :sepal-width
                         :=y :sepal-length
                         :=color :species
                         :=mark-size 10}))

## :kind/table


In [ ]:
(kind/table {:column-names [:a :b] :row-vectors [[1 2]]})

table header not working ...


In [ ]:
(kind/table
 {:column-names [(kind/hiccup
                  [:div {:style {:background-color "#ccaabb"}} [:big ":x"]])
                 (kind/hiccup
                  [:div {:style {:background-color "#aabbcc"}} [:big ":y"]])]
  :row-vectors [[ "*some text* **some more text**"
                 (kind/code "{:x (1 2 [3 4])}")]
                [(tc/dataset {:x (range 3)
                              :y (map inc (range 3))})
                 (p 9)]
                [(kind/plotly plotly-example)
                 (kind/md "$x^2$")]]})

## most kinds (but not all) can be nested inside a few containers

### in hiccup

In [ ]:

(kind/hiccup [:ul 
              [:li 1] 
              [:li (p 10)] 
              [:li 3]
              [:li image]])

### in seq
    

In [ ]:
[(p 1) (p 5) (p 10)]

## kinds can as well be on vars

In this example the "kind" is not medatadata of the value, but metadata of the var.

In [ ]:
(def iris-no-meta
 (with-meta
   (-> iris
    (hanami/plot hanami/point-chart
                 {:=x :sepal-width
                  :=y :sepal-length
                  :=color :species
                  :=mark-size 200}))
    {}))

In [ ]:
^kind/vega-lite iris-no-meta